In [5]:
# ** CHEQUEAR QUÉ LIBRERIAS ESTOY Y NO ESTOY USANDO** 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#dust_file='Dust records.xlsx'
dust_file='Dust records iniciales.xlsx'
temp_file='Temps records.xlsx'
id_columns=['Name','Data id','Latitud','Longitud','Age units','Data units','Data length']

In [6]:
id_data=pd.read_excel(dust_file,sheet_name=0,usecols=id_columns)
df_data=pd.read_excel(dust_file,sheet_name=1,skiprows=1)

df_data.head()

,Year,DustFlux,Year.1,DustFlux.1,Year.2,nssCa2 flux,Year.3,Ca2+,Year.4,Th norm Litho flux,...,Year.10,DustFlux.5,Year.11,MAR,Year.12,DustConc,Year.13,Ca2+.1,Year.14,Part conc
0,4.509,602.784,0.375,0.259,1.19948,112.4,3.791,0.2,14.424956,0.091053,...,1.15,238095.24,0.0,10.72,5.00,305.363666,7.8006,12.8,10.1479,60.539
1,4.552,423.903,0.400,0.329,1.21246,125.3,3.791,0.0,19.748589,0.168297,...,2.20,238095.24,2.0,10.40,5.05,471.395055,7.8058,13.2,10.1555,73.720
2,4.690,299.095,0.425,0.652,1.22582,380.3,3.792,NaN,23.741313,0.192389,...,3.25,277777.78,4.0,8.57,5.10,266.747998,7.8111,13.6,10.1631,52.932
3,5.259,734.449,0.450,0.341,1.23938,150.0,3.792,0.8,29.730400,0.185395,...,4.15,265957.45,6.0,7.80,5.15,252.657484,7.8163,17.2,10.1707,76.469
4,5.674,603.242,0.475,0.312,1.25299,0.0,3.792,1.3,37.050396,0.150754,...,5.09,263157.89,8.0,7.98,5.20,338.667402,7.8215,16.4,10.1781,56.083


In [7]:
#SEPARAR CADA UNA DE LAS COLUMNAS v.2
columns=df_data.size/len(df_data) #data.size = nxm ; len(data) = n -> cuociente = m = n°columnas
lis=[] #Lista de dupla de dataframes: Age & Values
new_length=np.array([])

for i in range(int(columns)):
    if (i+1)%2==1: #División completa, voy de 2 en 2
        dupla=df_data[[df_data.columns[i],df_data.columns[i+1]]][:id_data['Data length'][int(i/2)]]
        dupla.index=[j for j in range(0, len(dupla))] 
        lis.append(dupla)
        new_length=np.append(new_length,np.shape(dupla)[0])

In [8]:
#Función para ver si hay años repetidos
def diff(ages):
    return np.array(ages[:-1])-np.array(ages[1:])

bases_a_promediar=[]
for i in range(len(lis)):
    diferencias=diff(lis[i][lis[i].columns[0]])
    if 0 in diferencias:
        bases_a_promediar.append(i)
        print(id_data['Name'][i],' tiene años repetidos')

print('\nPromediar las bases',bases_a_promediar)

Talos Dome  tiene años repetidos
ML1208-17PC  tiene años repetidos
ML1208-31BB  tiene años repetidos
ML1208-37BB  tiene años repetidos

Promediar las bases [3, 7, 8, 9]


In [15]:
# Defino la base que voy a reparar:
base=lis[15] 

In [16]:
#Promedio los años repetidos
years=[]
new_age=np.array([])
new_values=np.array([])
for year in base[base.columns[0]]:
    if year not in years:
        indices=base.index[(base[base.columns[0]] ==year)].tolist()#índices del año repetido en evaluación
        valid_interval=len(indices)-np.isnan(base[base.columns[1]][indices]).sum()
        if valid_interval!=0:
            new_values=np.append(new_values,base[base.columns[1]][indices].mean())
            #print('El intervalo ',year,' presenta solo nans')
        else:
            new_values=np.append(new_values,np.nan)
        
        years.append(year)
        
new_age=np.array(years)


TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [79]:
anos_promediados = pd.DataFrame(new_age, columns = ['Year'])
valores_promediados = pd.DataFrame(new_values, columns = ['Values'])
base_corregida=pd.concat([anos_promediados,valores_promediados], axis=1)

base_corregida.to_excel(id_data['Name'][6]+' corregido-promediado.xlsx')

In [80]:
base_corregida

,Year,Values
0,-0.017,-9.50
1,0.000,-9.66
2,0.020,-9.58
3,0.038,-9.50
4,0.056,-9.33
...,...,...
6106,640.300,-8.86
6107,640.500,-9.14
6108,640.800,-9.06
6109,641.000,-9.34


In [81]:
len(base)

8353